In [2]:
import multiprocessing
import os
from io import BytesIO
from urllib3 import request
import pandas as pd
import re
import tqdm
import requests as req
from PIL import Image
from io import BytesIO

In [3]:
#set files and dir
train_df = pd.read_csv('train.csv')
# preferences
TARGET_SIZE = 128
IMG_QUALITY = 90  
NUM_WORKERS = 8  

In [6]:
test_download = train_df.loc[:100,]
test_download

,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231
5,2c9c54b62f0a6a37,https://lh5.googleusercontent.com/-mEaSECO7D-4...,10400
6,0aac70a1de44ced6,http://lh6.ggpht.com/-cJMh9AYQGk8/SOkF_Q5PrjI/...,9779
7,de770bc720f8e714,https://lh4.googleusercontent.com/-Q_FvRlwaaa8...,11288
8,dc9457d703e612ad,https://lh3.googleusercontent.com/-Px33Q-wekRI...,13170
9,3060f5f75d936abb,http://lh3.ggpht.com/-KXyELwqwp_Q/Ry-qmQAqwUI/...,6051


In [4]:
for i in range(len(test_download)):
    filename = "./train/test_{}.jpeg".format(str(i))
    print(filename)
    path = test_download.url[i]
    print(path)
    response = req.get(path)
    pil_image = Image.open(BytesIO(response.content))
    pil_image_rgb = pil_image.convert('RGB')
    pil_image_resize = pil_image_rgb.resize((TARGET_SIZE, TARGET_SIZE))
    pil_image_resize.save(filename, quality=IMG_QUALITY)

./train/test_0.jpeg
http://static.panoramio.com/photos/original/70761397.jpg
./train/test_1.jpeg
http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/AAAAAAAAAE4/cDiNGkoQX88/s1600/
./train/test_2.jpeg
http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/AAAAAAAAAco/-7JrhF1dfso/s1600/
./train/test_3.jpeg
https://lh3.googleusercontent.com/-LOW2cjAqubA/RvE11dfgUaI/AAAAAAAABoU/ItwXEejtwHg/rj/
./train/test_4.jpeg
https://lh5.googleusercontent.com/-tnmSXwQcWL8/SfhIj_3pugI/AAAAAAAAAdg/X55Wx8dGz-Q/s1600/


In [17]:
def image_download(df):
    global error_download
    try:
        for i in range(len(df)):
            filename = "./train/{}.jpeg".format(str(df.id[i]))
            print('Detect the image id and generate filename:' + filename)
            path = df.url[i]
            print('Detect the image and download from' + path)
            response = req.get(path)
            pil_image = Image.open(BytesIO(response.content))
            pil_image_rgb = pil_image.convert('RGB')
            pil_image_resize = pil_image_rgb.resize((TARGET_SIZE, TARGET_SIZE))
            pil_image_resize.save(filename, quality=IMG_QUALITY)
    except:
        print('Fail to download the image.')
        error_download += 1
        pass

In [19]:
image_download(test_download)

Detect the image id and generate filename:./train/cacf8152e2d2ae60.jpeg
Detect the image and download fromhttp://static.panoramio.com/photos/original/70761397.jpg
Detect the image id and generate filename:./train/0a58358a2afd3e4e.jpeg
Detect the image and download fromhttp://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/AAAAAAAAAE4/cDiNGkoQX88/s1600/
Detect the image id and generate filename:./train/6b2bb500b6a38aa0.jpeg
Detect the image and download fromhttp://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/AAAAAAAAAco/-7JrhF1dfso/s1600/
Detect the image id and generate filename:./train/b399f09dee9c3c67.jpeg
Detect the image and download fromhttps://lh3.googleusercontent.com/-LOW2cjAqubA/RvE11dfgUaI/AAAAAAAABoU/ItwXEejtwHg/rj/
Detect the image id and generate filename:./train/19ace29d77a5be66.jpeg
Detect the image and download fromhttps://lh5.googleusercontent.com/-tnmSXwQcWL8/SfhIj_3pugI/AAAAAAAAAdg/X55Wx8dGz-Q/s1600/
